In [ ]:
import numpy as np
import csv
import time
import random
import matplotlib.pylab as plt
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt
import pandas as pd
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error
from math import sqrt


In [ ]:
df = pd.read_csv('Bitcoin.csv')
df['Date'] = pd.to_datetime(df.Date)
df = df.sort_values(by='Date')
df.set_index('Date', inplace=True) #set Date as index
train = df[0:1500]
test = df[1500:]

In [ ]:
ax = train.plot( y = 'Close', figsize=(15,8), title= 'Bitcoin Price', fontsize=14, label = '2014-2018', color = "black")
test.plot(ax=ax, y = 'Close' ,figsize=(15,8), title= 'Bitcoin Price', fontsize=14, label = '2018-2019', color = "orange")
plt.show()


In [ ]:
dd = np.asarray(train.Close)
y_hat = test.copy()
y_hat['naive'] = dd[len(dd)-1]
ax = train.plot(y = 'Close', figsize=(15,8), title= 'Bitcoin Price', fontsize=14, label = '2014-2018', color = "black")
test.plot(ax=ax,  y = 'Close' ,figsize=(15,8), title= 'Bitcoin Price', fontsize=14, label = '2018-2019', color = "orange")
y_hat.plot(ax=ax, y = 'naive', figsize=(15,8), title= 'Bitcoin Price', fontsize=14, label = 'Predicted', color = "blue")
plt.show()
rms = sqrt(mean_squared_error(test.Close, y_hat.naive))
print(rms)


In [ ]:
dd = np.asarray(train.Close)
y_hat = test.copy()
y_hat['avg_forecast'] = train['Close'].mean()
ax = train.plot(y = 'Close', figsize=(15,8), title= 'Bitcoin Price', fontsize=14, label = '2014-2018', color = "black")
test.plot(ax=ax,y = 'Close' ,figsize=(15,8), title= 'Bitcoin Price', fontsize=14, label = '2018-2019', color = "orange")
y_hat.plot(ax=ax, y = 'avg_forecast', figsize=(15,8), title= 'Bitcoin Price', fontsize=14, label = 'Predicted', color = "blue")
plt.show()
rms = sqrt(mean_squared_error(test.Close, y_hat.avg_forecast))
print(rms)


In [ ]:
dd = np.asarray(train.Close)
y_hat= test.copy()
y_hat['moving_avg_forecast'] = train['Close'].rolling(7).mean().iloc[-1]
ax = train.plot(y = 'Close', figsize=(15,8), title= 'Bitcoin Price', fontsize=14, label = '2014-2018', color = "black")
test.plot(ax=ax, y = 'Close' ,figsize=(15,8), title= 'Bitcoin Price', fontsize=14, label = '2018-2019', color = "orange")
y_hat.plot(ax=ax, y = 'moving_avg_forecast', figsize=(15,8), title= 'Bitcoin Price', fontsize=14, label = 'Predicted', color = "blue")
plt.show()
rms = sqrt(mean_squared_error(test.Close, y_hat.moving_avg_forecast))
print(rms)

In [ ]:
y_hat = test.copy()
fit2 = SimpleExpSmoothing(np.asarray(train['Close'])).fit(smoothing_level=0.6,optimized=False)
y_hat['SES'] = fit2.forecast(len(test))
ax = train.plot(y = 'Close', figsize=(15,8), title= 'Bitcoin Price', fontsize=14, label = '2014-2018', color = "black")
test.plot(ax=ax, y = 'Close' ,figsize=(15,8), title= 'Bitcoin Price', fontsize=14, label = '2018-2019', color = "orange")
y_hat.plot(ax=ax,y = 'SES', figsize=(15,8), title= 'Bitcoin Price', fontsize=14, label = 'Predicted', color = "blue")
plt.show()
rms = sqrt(mean_squared_error(test.Close, y_hat.SES))
print(rms)

In [ ]:
y_hat = test.copy()
fit1 = Holt(np.asarray(train['Close'])).fit(smoothing_level = 0.3,smoothing_slope = 0.61)
y_hat['Holt_linear'] = fit1.forecast(len(test))
ax = train.plot(y = 'Close', figsize=(15,8), title= 'Bitcoin Price', fontsize=14, label = '2014-2018', color = "black")
test.plot(ax=ax, y = 'Close' ,figsize=(15,8), title= 'Bitcoin Price', fontsize=14, label = '2018-2019', color = "orange")
y_hat.plot(ax=ax, y = 'Holt_linear', figsize=(15,8), title= 'Bitcoin Price', fontsize=14, label = 'Predicted', color = "blue")
plt.show()
rms = sqrt(mean_squared_error(test.Close, y_hat.Holt_linear))
print(rms)


In [ ]:
y_hat = test.copy()
fit1 = ExponentialSmoothing(np.asarray(train['Close']) ,seasonal_periods=70 ,trend='add', seasonal='add',).fit()
y_hat['Holt_Winter'] = fit1.forecast(len(test))
ax = train.plot( y = 'Close', figsize=(15,8), title= 'Bitcoin Price', fontsize=14, label = '2014-2018', color = "black")
test.plot(ax=ax,  y = 'Close' ,figsize=(15,8), title= 'Bitcoin Price', fontsize=14, label = '2018-2019', color = "orange")
y_hat.plot(ax=ax,  y = 'Holt_Winter', figsize=(15,8), title= 'Bitcoin Price', fontsize=14, label = 'Predicted', color = "blue")
plt.show()
rms = sqrt(mean_squared_error(test.Close, y_hat.Holt_Winter))
print(rms)

In [ ]:

y_hat = test.copy()
fit1 = sm.tsa.statespace.SARIMAX(train.Close, order = (2,1,4), seasonal_order= (0,1,0,60),  enforce_stationarity=False,
                                enforce_invertibility=False).fit()
y_hat['SARIMA'] = fit1.predict(start='2018-02-09', end='2018-12-22', dynamic=True)
ax = train.plot( y = 'Close', figsize=(15,8), title= 'Bitcoin Price', fontsize=14, label = '2014-2018', color = "black")
test.plot(ax=ax,y = 'Close' ,figsize=(15,8), title= 'Bitcoin Price', fontsize=14, label = '2018-2019', color = "orange")
y_hat.plot(ax=ax, y = 'SARIMA', figsize=(15,8), title= 'Bitcoin Price', fontsize=14, label = 'Predicted', color = "blue")
plt.show()
rms = sqrt(mean_squared_error(test.Close, y_hat.SARIMA))
print(rms)


In [ ]:
fig = plt.figure(figsize=(12,8))
ax1 = fig.add_subplot(211)
fig = sm.graphics.tsa.plot_acf(train['Close'].diff().dropna(), lags=40, ax=ax1)

In [ ]:
sm.tsa.seasonal_decompose(train.Close).plot()
result = sm.tsa.stattools.adfuller(train.Close)
plt.show()